In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
339,tt1232207,20000000,17436509,Capitalism: A Love Story,Michael Moore|Thora Birch|William Black|Elijah...,Michael Moore,"Ask not what your C.E.O. can do for you, but w...",Michael Moore's Capitalism: A Love Story comes...,120,Documentary,The Weinstein Company|Paramount Vantage,9/6/2009,6.9,2009
538,tt0159273,40000000,91753202,Behind Enemy Lines,Owen Wilson|Gene Hackman|Gabriel Macht|Charles...,John Moore,His only weapon is his will to survive.,While flying a routine reconnaissance mission ...,106,Action|Drama|Thriller|War,Twentieth Century Fox Film Corporation|Davis E...,11/17/2001,5.9,2001
1250,tt1924429,20000000,24261569,Trance,James McAvoy|Vincent Cassel|Rosario Dawson|Mat...,Danny Boyle,Don't be a hero.,"A fine art auctioneer mixed up with a gang, jo...",101,Thriller|Crime|Drama|Mystery,Cloud Eight Films|PathÃ©|Decibel Films|Film4,3/27/2013,6.4,2013
436,tt0964517,25000000,9190869,The Fighter,Mark Wahlberg|Christian Bale|Amy Adams|Melissa...,David O. Russell,Every dream deserves a fighting chance.,"The Fighter, is a drama about boxer ""Irish"" Mi...",116,Drama,Paramount Pictures,12/17/2010,7.1,2010
383,tt1542344,18000000,35692920,127 Hours,James Franco|Kate Mara|Amber Tamblyn|ClÃ©mence...,Danny Boyle,There is no force more powerful than the will ...,127 Hours is the true story of mountain climbe...,94,Adventure|Drama|Thriller,Fox Searchlight Pictures|Dune Entertainment|Da...,11/5/2010,6.9,2010


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

In [6]:
#добавим столбец 'profit'
data['profit'] = data['revenue'] - data['budget']
# добавим столбец 'month' (так как используем 2 раза)
data['month'] = pd.to_datetime(data.release_date).dt.month

In [7]:
def list_sort(incoming_list):
    """Функция принимает список и выдает словарь 
    {ключ (элемент словаря): значение (сколько раз встречается)}
    """
    sort_dict = {}
    
    for elem in incoming_list:
        if elem in sort_dict:
            sort_dict[elem] += 1
        else:
            sort_dict[elem] = 1
    
    return sort_dict
        
            

In [8]:
#Столбцы с разделителем '|' переделаем в список, чтоб потом каждый раз этого не делать
data.genres = data.genres.apply(lambda s: s.split('|'))#жанры
data.cast = data.cast.apply(lambda s: s.split('|'))# актеры
data.production_companies = data.production_companies.apply(lambda s: s.split('|'))#компании
data.director = data.director.apply(lambda s: s.split('|'))#режиссеры

# Проверяем, что вышло
data.head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]","[Universal Studios, Amblin Entertainment, Lege...",6/9/2015,6.5,2015,1363528810,6
1,tt1392190,150000000,378436354,Mad Max: Fury Road,"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...",[George Miller],What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,"[Action, Adventure, Science Fiction, Thriller]","[Village Roadshow Pictures, Kennedy Miller Pro...",5/13/2015,7.1,2015,228436354,5
2,tt2908446,110000000,295238201,Insurgent,"[Shailene Woodley, Theo James, Kate Winslet, A...",[Robert Schwentke],One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,"[Adventure, Science Fiction, Thriller]","[Summit Entertainment, Mandeville Films, Red W...",3/18/2015,6.3,2015,185238201,3
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...",[J.J. Abrams],Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,"[Action, Adventure, Science Fiction, Fantasy]","[Lucasfilm, Truenorth Productions, Bad Robot]",12/15/2015,7.5,2015,1868178225,12
4,tt2820852,190000000,1506249360,Furious 7,"[Vin Diesel, Paul Walker, Jason Statham, Miche...",[James Wan],Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,"[Action, Crime, Thriller]","[Universal Pictures, Original Film, Media Righ...",4/1/2015,7.3,2015,1316249360,4


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [9]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
#answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа

# если ответили верно, можете добавить комментарий со значком "+"

In [10]:
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [11]:
# тут пишем ваш код для решения данного вопроса:
data.sort_values('budget', ascending=False).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,"[Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...",[Rob Marshall],Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,"[Adventure, Action, Fantasy]","[Walt Disney Pictures, Jerry Bruckheimer Films...",5/11/2011,6.3,2011,641683000,5


ВАРИАНТ 2

In [12]:
data[data['budget'] == data['budget'].max()][['imdb_id', 'original_title', 'budget']]

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


# 2. Какой из фильмов самый длительный (в минутах)?

In [13]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '1157. Gods and Generals (tt0279111)'

In [14]:
data.sort_values('runtime', ascending=False).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month
1157,tt0279111,56000000,12923936,Gods and Generals,"[Stephen Lang, Jeff Daniels, Robert Duvall, Ke...",[Ronald F. Maxwell],The nations heart was touched by...,The film centers mostly around the personal an...,214,"[Drama, History, War]","[Turner Pictures, Antietam Filmworks]",2/21/2003,5.8,2003,-43076064,2


# 3. Какой из фильмов самый короткий (в минутах)?





In [15]:
answers['3'] = '768. Winnie the Pooh (tt1449283)'

In [16]:
data.sort_values('runtime').head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month
768,tt1449283,30000000,14460000,Winnie the Pooh,"[Jim Cummings, Travis Oates, Jim Cummings, Bud...","[Stephen Anderson, Don Hall]",Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,"[Animation, Family]","[Walt Disney Pictures, Walt Disney Animation S...",4/13/2011,6.8,2011,-15540000,4


In [17]:
# Или так:
data[data['runtime'] == data['runtime'].min()][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


# 4. Какова средняя длительность фильмов?


In [18]:
answers['4'] = 110

In [19]:
round(data['runtime'].mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [20]:
answers['5'] = 107

In [21]:
round(data['runtime'].median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [22]:
answers['6'] = '239. Avatar (tt0499549)'

In [23]:
# лучше код получения столбца profit вынести в Предобработку что в начале

In [24]:
data.sort_values('profit', ascending=False)[['profit','budget', 'revenue', 'original_title', 'imdb_id']].head(1)

,profit,budget,revenue,original_title,imdb_id
239,2544505847,237000000,2781505847,Avatar,tt0499549


# 7. Какой фильм самый убыточный? 

In [25]:
answers['7'] = '1245. The Lone Ranger (tt1210819)'

In [26]:
data.sort_values('profit')[['profit','budget', 'revenue', 'original_title', 'imdb_id']].head(1)

,profit,budget,revenue,original_title,imdb_id
1245,-165710090,255000000,89289910,The Lone Ranger,tt1210819


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [27]:
answers['8'] = 1478

In [28]:
data[data['budget'] < data['revenue']]['imdb_id'].count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [29]:
answers['9'] = '599. The Dark Knight (tt0468569)'

In [30]:
data[data['release_year'] == 2008].sort_values('revenue', ascending=False)[['revenue', 'original_title', 'imdb_id']].head(1)

,revenue,original_title,imdb_id
599,1001921825,The Dark Knight,tt0468569


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [31]:
answers['10'] = '1245. The Lone Ranger (tt1210819)'

In [32]:
data[(data['release_year'] >= 2012) & (data['release_year'] <= 2014)]\
.sort_values('profit')[['profit', 'original_title', 'imdb_id', 'release_year']].head(1)

,profit,original_title,imdb_id,release_year
1245,-165710090,The Lone Ranger,tt1210819,2013


# 11. Какого жанра фильмов больше всего?

In [33]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [34]:
answers['11'] = 'Drama'

In [35]:
#Вариант 1
data[['genres']].explode('genres').value_counts()

genres         
Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

ВАРИАНТ 2

In [36]:
# колонку преобразуем в список и сразу его обрабатываем 
genres_list = [elem for el1 in list(data['genres']) for elem in el1] 

# прогоняем через функцию (написана мной и вынесена вверх) наш подготовленный список
genres_dict = list_sort(genres_list)
print(genres_dict) #смотрим, что получилось

sorted(genres_dict, key=genres_dict.get, reverse=True)# После сортировки верхний вариант - наш

{'Action': 582, 'Adventure': 415, 'Science Fiction': 248, 'Thriller': 596, 'Fantasy': 222, 'Crime': 315, 'Western': 19, 'Drama': 782, 'Family': 260, 'Animation': 139, 'Comedy': 683, 'Mystery': 168, 'Romance': 308, 'War': 58, 'History': 62, 'Music': 64, 'Horror': 176, 'Documentary': 8, 'Foreign': 2}


['Drama',
 'Comedy',
 'Thriller',
 'Action',
 'Adventure',
 'Crime',
 'Romance',
 'Family',
 'Science Fiction',
 'Fantasy',
 'Horror',
 'Mystery',
 'Animation',
 'Music',
 'History',
 'War',
 'Western',
 'Documentary',
 'Foreign']

In [37]:
# вариант 3
Counter(genres_list).most_common() 

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [38]:
answers['12'] = 'Drama'

In [39]:
# Так как работаем с отдельными колонками, то делаем отбор колонок в отдельный датафрейм. 
# В дальнейшем такой схемы будем придерживаться (по надобности)

data_v12 = data[['genres', 'profit']].explode('genres')

# Просто часто встречающееся значение
data_v12[data_v12['profit'] > 0]['genres'].mode()

0    Drama
dtype: object

In [40]:
# или сортировка с данными о количестве 
data_v12[data_v12['profit'] > 0].groupby(['genres']).nunique().sort_values(by = 'profit', ascending = False )

,profit
genres,
Drama,559
Comedy,550
Thriller,445
Action,444
Adventure,337
Romance,242
Crime,231
Family,226
Science Fiction,195


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [41]:
answers['13'] = 'Peter Jackson'

In [42]:
data_v13 = data[['director','revenue']].explode('director')

data_v13.groupby('director').revenue.sum().sort_values(ascending=False).head(3)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [43]:
answers['14'] = 'Robert Rodriguez'

In [44]:
data_v14 = data[['director','genres']].explode('director').explode('genres')

data_v14[data_v14['genres'] == 'Action']['director'].value_counts().head(3)

Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Name: director, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [45]:
answers['15'] = 'Chris Hemsworth' 

In [46]:
data_v15 = data[['cast', 'revenue', 'release_year']].explode('cast')

data_v15[data_v15['release_year'] == 2012].groupby('cast').revenue.sum().sort_values(ascending=False).head(3)

cast
Chris Hemsworth    2027450773
Denis Leary        1629460639
Anne Hathaway      1522851057
Name: revenue, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [47]:
answers['16'] = 'Matt Damon' 

In [48]:
#Примечание: в фильмах, где бюджет выше среднего по данной выборке.
data_v16 = data[['cast', 'budget']].explode('cast')

data_v16[data_v16['budget'] > data_v16['budget'].mean()].cast.mode()

0    Matt Damon
dtype: object

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [49]:
answers['17'] = 'Action' 

In [50]:
data_v17 = data[['cast', 'genres']].explode('cast').explode('genres')

data_v17[data_v17['cast'] == 'Nicolas Cage'].genres.mode()

0    Action
dtype: object

In [51]:
#Если нам надо не просто узнать больше, а посмотреть распределение и количество фильмов
data_v17[data_v17['cast'] == 'Nicolas Cage'].groupby(['genres']).count().sort_values(by = 'cast', ascending = False)

,cast
genres,
Action,17
Thriller,15
Drama,12
Crime,10
Fantasy,8
Adventure,7
Comedy,6
Science Fiction,4
Animation,3


# 18. Самый убыточный фильм от Paramount Pictures

In [52]:
answers['18'] = '925. K-19: The Widowmaker (tt0267626)'

In [53]:
data_v18 = data[['production_companies', 'original_title', 'imdb_id', 'profit']].explode('production_companies')

data_v18[data_v18['production_companies'] == 'Paramount Pictures'].sort_values(by = 'profit').head(3)

,production_companies,original_title,imdb_id,profit
925,Paramount Pictures,K-19: The Widowmaker,tt0267626,-64831034
1123,Paramount Pictures,Timeline,tt0300556,-60519261
1722,Paramount Pictures,Next,tt0435705,-51788987


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [54]:
answers['19'] = 2015

In [55]:
data_v19 = data[['release_year', 'revenue']]

data_v19.groupby(['release_year']).sum().sort_values(by = 'revenue', ascending = False)

,revenue
release_year,
2015,25449202382
2014,23405862953
2013,23213799791
2012,23079001687
2011,22676791872
2010,21071204961
2009,20261791024
2008,18252781990
2007,18162406801


# 20. Какой самый прибыльный год для студии Warner Bros?

In [56]:
answers['20'] = 2014

In [57]:
data_v20 = data[['production_companies', 'profit', 'release_year']].explode('production_companies')

data_v20[data_v20['production_companies'].str.contains('Warner Bros')].groupby(['release_year'])['profit'].sum()\
.sort_values(ascending = False)


release_year
2014    2292949646
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1844008221
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1282142579
2012    1258020056
2002    1022709901
2015     870368348
2006     625351872
2000     452631386
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [58]:
answers['21'] = '9 - сентябрь'

In [59]:
data['month'].value_counts()

9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: month, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [60]:
answers['22'] = 450

In [61]:
data[(data['month'] >= 6) & (data['month'] <= 8)]['imdb_id'].count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [62]:
answers['23'] = 'Peter Jackson'

In [63]:
data_v23 = data[['imdb_id', 'director', 'month']].explode('director')

data_v23[(data_v23['month'] <=2 ) | (data_v23['month'] == 12)].groupby('director').count()\
.sort_values(by = 'imdb_id', ascending = False)

,imdb_id,month
director,,
Peter Jackson,7,7
Steven Soderbergh,6,6
Clint Eastwood,6,6
Shawn Levy,4,4
Adam Shankman,4,4
...,...,...
John Luessenhop,1,1
John Musker,1,1
John Polson,1,1


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [64]:
answers['24'] = 'Four By Two Productions & Twentieth Century Fox Film Corporation'

In [65]:
data_v24 = data[['production_companies', 'original_title']].explode('production_companies')

# Добавим столбец длина названия фильма
data_v24['len_original_title'] = data_v24.original_title.apply(lambda s: len(s))

data_v24.groupby(['production_companies']).len_original_title.max().sort_values(ascending = False).head(3)

production_companies
Twentieth Century Fox Film Corporation    83
Four By Two Productions                   83
Walt Disney                               62
Name: len_original_title, dtype: int64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [66]:
answers['25'] = 'Midnight Picture Show'

In [67]:
data_v25 = data[['production_companies', 'overview']].explode('production_companies')

# Добавим столбец длина описания фильма
data_v25['len_overview'] = data_v25.overview.apply(lambda s: len(s))

data_v25.groupby(['production_companies']).len_overview.mean().sort_values(ascending = False).head(3)

production_companies
Midnight Picture Show               1000.0
Room 9 Entertainment                 964.0
Brookwell-McNamara Entertainment     936.0
Name: len_overview, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [68]:
answers['26'] = ['12 Years a Slave',
                 'Big Hero 6',
                 'Gone Girl',
                 'Guardians of the Galaxy',
                 'Her',
                 'Inception',
                 'Inside Out',
                 'Interstellar',
                 'Memento',
                 'Room',
                 'Spotlight',
                 'The Dark Knight',
                 'The Fault in Our Stars',
                 'The Grand Budapest Hotel',
                 'The Imitation Game',
                 'The Lord of the Rings: The Return of the King',
                 'The Lord of the Rings: The Two Towers',
                 'The Pianist',
                 'The Wolf of Wall Street']

In [69]:
#Находим какое количество фильмов будет составлять 1% от общего количества фильмов
amount_movies = round(data.shape[0]/100)
the_best_movies = data.sort_values(by = 'vote_average', ascending = False).head(amount_movies)

#Создаем отсортированный по имени список рейтинговых фильмов, а то искать неудобно 
movis_list = sorted(list(the_best_movies['original_title']))
movis_list

['12 Years a Slave',
 'Big Hero 6',
 'Gone Girl',
 'Guardians of the Galaxy',
 'Her',
 'Inception',
 'Inside Out',
 'Interstellar',
 'Memento',
 'Room',
 'Spotlight',
 'The Dark Knight',
 'The Fault in Our Stars',
 'The Grand Budapest Hotel',
 'The Imitation Game',
 'The Lord of the Rings: The Return of the King',
 'The Lord of the Rings: The Two Towers',
 'The Pianist',
 'The Wolf of Wall Street']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [70]:
answers['27'] = ['Rupert Grint - Daniel Radcliffe',
                'Emma Watson - Daniel Radcliffe',
                'Emma Watson - Rupert Grint']

In [71]:
# Создаем 2 копии датафреймов с нужными столбцами
data_v27_1 = data[['imdb_id', 'cast']]
data_v27_2 = data[['imdb_id', 'cast']]

# Меняем название колонок актеров
data_v27_1 = data_v27_1.rename(columns = {'cast':'cast_1'})
data_v27_2 = data_v27_2.rename(columns = {'cast':'cast_2'})

# Склеиваем два этих датафрейма
data_v27 = data_v27_1.merge(data_v27_2, on = 'imdb_id', how = 'left')

# Раскладываем списки актеров в строки
data_v27 = data_v27.explode('cast_1').explode('cast_2')

# Убираем строки, где актер играет сам с собой
data_v27 = data_v27[data_v27['cast_1'] != data_v27['cast_2']]

# В cast_3 собираем пары актеров в отдельный столбец
data_v27['cast_3'] = data_v27['cast_1'] +' - '+ data_v27['cast_2']

# Группируем и получаем сколько раз пара сыграла вместе (можно смотреть(актер_1 - актер_2) и (актер_2 - актер_1)
# Это дает, при необходимости, сделать сортировку пар по именам - каждый актер будет впереди
data_v27.groupby(['cast_3']).count().sort_values(['imdb_id'], ascending = False).head(10)


,imdb_id,cast_1,cast_2
cast_3,,,
Rupert Grint - Daniel Radcliffe,8,8,8
Emma Watson - Daniel Radcliffe,8,8,8
Emma Watson - Rupert Grint,8,8,8
Daniel Radcliffe - Emma Watson,8,8,8
Rupert Grint - Emma Watson,8,8,8
Daniel Radcliffe - Rupert Grint,8,8,8
Johnny Depp - Helena Bonham Carter,6,6,6
Owen Wilson - Ben Stiller,6,6,6
Helena Bonham Carter - Johnny Depp,6,6,6


ВАРИАНТ 2

# Submission

In [72]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157. Gods and Generals (tt0279111)',
 '3': '768. Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': '239. Avatar (tt0499549)',
 '7': '1245. The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': '599. The Dark Knight (tt0468569)',
 '10': '1245. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': '925. K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': '9 - сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions & Twentieth Century Fox Film Corporation',
 '25': 'Midnight Picture Show',
 '26': ['12 Years a Slave',
  'Big Hero 6',
  'Gone Girl',
  'Guardians of the Galaxy',
  'Her',
  'Inception',
  'Inside Out',
  'Interstellar',
  'Memento',
  'Room',
  'Spotlight',
  'The Dark Knight',
  'The Fault in Our Stars',
  'The Grand Budapest Hotel

In [73]:
# и убедиться что ни чего не пропустил)
len(answers)

27